<a href="https://colab.research.google.com/github/laysfreitas/Covid_Dados/blob/main/LSTM_Covid_Data_Obitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importações**

In [46]:
import requests
import csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# **Carregando os dados**

In [47]:
!gdown 'https://drive.google.com/uc?id=1Zi5iwuWt0fsH0uVOiyWZHctUX0pV_De7'

Downloading...
From: https://drive.google.com/uc?id=1Zi5iwuWt0fsH0uVOiyWZHctUX0pV_De7
To: /content/R_GO.csv
100% 861/861 [00:00<00:00, 2.34MB/s]


In [48]:
DF_Vacinacao = pd.read_csv('V_GO.csv')

In [49]:
DF = pd.read_csv('/content/R_GO.csv',sep=',')

In [50]:
DF.head()

,Unnamed: 0,saidaConfirmadaObitos,casosNovos
0,Janeiro 2020,0.0,0.0
1,Fevereiro 2020,0.0,0.0
2,Março 2020,2.0,130.0
3,Abril 2020,56.0,1432.0
4,Maio 2020,190.0,5842.0


In [51]:
DF_Vacinacao.head()

,Unnamed: 0,1ª Dose,2ª Dose,Dose Adicional,Reforço,2ª Dose Revacinação,Dose,1ª Dose Revacinação,2º Reforço,Revacinação,3ª Dose Revacinação,3º Reforço,4ª Dose Revacinação,Dose Inicial,3ª Dose,5ª Dose
0,Janeiro-2021,31832.0,57.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Fevereiro-2021,43821.0,23495.0,5.0,3.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Março-2021,141303.0,29779.0,11.0,11.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abril-2021,112493.0,129479.0,4.0,4.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Maio-2021,183377.0,43527.0,0.0,5.0,0.0,4.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# **Preparando os dados**

In [52]:
DF_Vacinacao = DF_Vacinacao.iloc[:,[0,1,2,4]]

In [53]:
DF_Vacinacao['Doses'] = DF_Vacinacao['1ª Dose'] + DF_Vacinacao['2ª Dose'] + DF_Vacinacao['Reforço']

<ipython-input-53-7004aa1ac1d1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_Vacinacao['Doses'] = DF_Vacinacao['1ª Dose'] + DF_Vacinacao['2ª Dose'] + DF_Vacinacao['Reforço']


In [54]:
DF_Vacinacao['Unnamed: 0'] = DF_Vacinacao['Unnamed: 0'].str.replace("-"," ")

<ipython-input-54-ceb755af3dfd>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_Vacinacao['Unnamed: 0'] = DF_Vacinacao['Unnamed: 0'].str.replace("-"," ")


In [55]:
DF_Vacinacao_2020 = pd.DataFrame({'Unnamed: 0': ['Janeiro 2020', 'Fevereiro 2020', 'Março 2020', 'Abril 2020','Maio 2020','Junho 2020','Julho 2020','Agosto 2020','Setembro 2020','Outubro 2020','Novembro 2020','Dezembro 2020'],
                                     '1ª Dose': [0,0,0,0,0,0,0,0,0,0,0,0],
                                     '2ª Dose': [0,0,0,0,0,0,0,0,0,0,0,0],
                                     'Reforço': [0,0,0,0,0,0,0,0,0,0,0,0],
                                     'Doses': [0,0,0,0,0,0,0,0,0,0,0,0]})

In [56]:
DF_Vac = pd.concat([DF_Vacinacao_2020, DF_Vacinacao])

In [57]:
DF_TOTAL = pd.merge(DF, DF_Vac, how = 'inner', on = 'Unnamed: 0')
DF_TOTAL.rename(columns={'Unnamed: 0': 'Data'}, inplace = True)
DF_TOTAL = DF_TOTAL.iloc[:,[1,2,6]]

In [58]:
DF_TOTAL

,saidaConfirmadaObitos,casosNovos,Doses
0,0.0,0.0,0.0
1,0.0,0.0,0.0
2,2.0,130.0,0.0
3,56.0,1432.0,0.0
4,190.0,5842.0,0.0
5,700.0,41358.0,0.0
6,2364.0,87120.0,0.0
7,2876.0,129130.0,0.0
8,3156.0,153810.0,0.0
9,2162.0,92072.0,0.0


# **Modelo LSTM**

In [63]:
# Dados de features X e o target y
X = DF_TOTAL[['casosNovos', 'Doses']].values
y = DF_TOTAL['saidaConfirmadaObitos'].values

# Conjunto de pontos passados para o modelo LSTM
window = 10

# Separar os dados em 80% treino e 20% teste
last = int(len(X) * 0.2)
X_train, X_test = X[:-last], X[-(last + window):]
y_train, y_test = y[:-last], y[-(last + window):]

# Armazenar window números de pontos em uma sequência
xin_train, next_y_train = [], []
for i in range(window, len(X_train)):
  xin_train.append(X_train[i-window:i])
  next_y_train.append(y_train[i])

# Transformar os dados para o formato LSTM
xin_train, next_y_train = np.array(xin_train), np.array(next_y_train)
xin_train = xin_train.reshape(xin_train.shape[0], xin_train.shape[1], X_train.shape[1])

# Para os dados de teste
xin_test, next_y_test = [], []
for i in range(window, len(X_test)):
  xin_test.append(X_test[i-window:i])
  next_y_test.append(y_test[i])

xin_test, next_y_test = np.array(xin_test), np.array(next_y_test)
xin_test = xin_test.reshape(xin_test.shape[0], xin_test.shape[1], X_test.shape[1])

Esse código cria um modelo Sequential de Rede Neural Artificial com duas camadas LSTM, com 50 unidades cada, seguidas por duas camadas de Dropout (com taxa de descarte de 0,2), e finalmente, uma camada densa com uma unidade. A função de perda utilizada é a "mean_squared_error" e o otimizador escolhido é o "adam". A forma da entrada da camada LSTM é definida como uma matriz tridimensional de formato (xin.shape[1], 1), onde xin.shape[1] representa o comprimento da janela de tempo.

In [64]:
# Keras é uma interface do TensorFlow
import tensorflow as tf
from sklearn.metrics import r2_score
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from keras.layers import Dropout

# Inicializar modelo LSTM

m = Sequential()
m.add(LSTM(units=50, return_sequences=True, input_shape=(xin.shape[1],1)))
m.add(Dropout(0.2))
m.add(LSTM(units=50))
m.add(Dropout(0.2))
m.add(Dense(units=1))
m.compile(optimizer='RMSprop', loss='mean_squared_logarithmic_error')



*   Mean Squared Error (MSE): A função MSE calcula a média dos erros ao quadrado entre as previsões e os valores reais. É amplamente utilizada em problemas de regressão.
*   Mean Squared Logarithmic Error (MSLE): A MSLE é semelhante à MSE, mas aplica a logaritmo aos erros antes de calcular a média. Isso é útil para problemas onde as diferenças entre as previsões e os valores reais são grandes.





* Adam: um otimizador baseado em gradiente que utiliza técnicas de adaptação de taxa de aprendizado
*  RMSprop: um otimizador baseado em gradiente que se concentra na descida de gradiente em direção às direções de descida de gradiente mais rápidas



In [66]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Construir o modelo LSTM
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(xin_train.shape[1], xin_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=1))

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Ajustar (fit) o modelo
model.fit(xin_train, next_y_train, epochs=100, batch_size=32)


Epoch 1/100
1/1 [==============================] - 5s 5s/step - loss: 8400232.0000
Epoch 2/100
1/1 [==============================] - 0s 23ms/step - loss: 8399618.0000
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 8398977.0000
Epoch 4/100
1/1 [==============================] - 0s 25ms/step - loss: 8398306.0000
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 8397693.0000
Epoch 6/100
1/1 [==============================] - 0s 25ms/step - loss: 8396994.0000
Epoch 7/100
1/1 [==============================] - 0s 20ms/step - loss: 8396334.0000
Epoch 8/100
1/1 [==============================] - 0s 20ms/step - loss: 8395658.0000
Epoch 9/100
1/1 [==============================] - 0s 21ms/step - loss: 8394942.0000
Epoch 10/100
1/1 [==============================] - 0s 28ms/step - loss: 8394228.0000
Epoch 11/100
1/1 [==============================] - 0s 24ms/step - loss: 8393320.0000
Epoch 12/100
1/1 [==============================] - 0s 29ms/step 

In [65]:
# Modelo LSTM fit
history = m.fit(xin, next_X, epochs = 50, batch_size = 50, verbose = 1)

plt.figure(figsize=(30.4,15.8))
plt.ylabel('loss'); plt.xlabel('epoch')
plt.semilogy(history.history['loss'])

ValueError: ignored

# **Validação**

In [ ]:
# Armazenar "window" pontos em uma sequência
xin = []
next_x1 = []

for i in range(window, len(Xtest)):
  xin.append(Xtest[i-window:i])
  next_x1.append(Xtest[i])

# Transformar os dados para o formato LSTM
xin, next_x1 = np.array(xin), np.array(next_x1)
xin = xin.reshape((xin.shape[0], xin.shape[1], 1))

# Predizer o próximo valor (1 passo a frente)
X_pred = m.predict(xin)

r2 = r2_score(next_x1, X_pred)
print(r2)

# Mostrar predição VS dados de teste atual
plt.figure(figsize=(30.4,15.8))
plt.plot(X_pred, ':', label = 'LSTM')
plt.plot(next_x1, '--', label='Atual')
plt.legend()

# **Forecasting**

Verificar o score R². Se o score R² for muito alto nos dados de treinamento, mas muito baixo nos dados de validação, é provável que o modelo esteja sofrendo de overfitting.

In [ ]:
# Usando valores preditivos para predizer o próximo passo

X_pred = Xtest.copy()
for i in range(window, len(X_pred)):
  xin = X_pred[i-window:i].reshape((1, window, 1))
  X_pred[i] = m.predict(xin)

r2 = r2_score(next_x1, X_pred[window:])
print(r2)

# Mostrar predição VS dados do teste atual
plt.figure(figsize=(30.4,15.8))
plt.plot(X_pred[window:], ':', label='LSTM')
plt.plot(next_x1, '--', label='Atual')
plt.legend()